In [2]:
pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 2.5 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.2 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=1b14f7ebbd41e6a9f4ebda282ecdae75e3fd00587eae10efbdfc9915c0c73c5f
  Stored in directory: /Users/datascientist/Library/Caches/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import numpy as np
import pretty_midi
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Directory paths
DATA_DIR = 'Composer_Dataset/NN_midi_files_extended'
DEV_DIR = os.path.join(DATA_DIR, 'dev')
TEST_DIR = os.path.join(DATA_DIR, 'test')
TRAIN_DIR = os.path.join(DATA_DIR, 'train')

In [3]:
# Load MIDI files and labels
def load_midi_files(directory):
    midi_files = []
    labels = []
    for composer in os.listdir(directory):
        composer_dir = os.path.join(directory, composer)
        if os.path.isdir(composer_dir):
            for file in os.listdir(composer_dir):
                if file.endswith('.mid') or file.endswith('.midi'):
                    file_path = os.path.join(composer_dir, file)
                    midi_data = pretty_midi.PrettyMIDI(file_path)
                    midi_files.append(midi_data)
                    labels.append(composer)
    return midi_files, labels

In [4]:
train_midi_files, train_labels = load_midi_files(TRAIN_DIR)
dev_midi_files, dev_labels = load_midi_files(DEV_DIR)
test_midi_files, test_labels = load_midi_files(TEST_DIR)

/opt/anaconda3/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [5]:
# Data augmentation
def augment_midi(midi_data):
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            note.pitch = note.pitch + np.random.choice([-1, 1])
    return midi_data

augmented_train_midi_files = [augment_midi(midi) for midi in train_midi_files]

In [6]:
# Extract features from MIDI files
def extract_features(midi_files):
    note_sequences = []
    chord_sequences = []
    tempo_sequences = []
    
    for midi in midi_files:
        notes = [note.pitch for instrument in midi.instruments for note in instrument.notes]
        chords = [note.start for instrument in midi.instruments for note in instrument.notes]
        tempos = midi.get_tempo_changes()[1]
        
        note_sequences.append(notes)
        chord_sequences.append(chords)
        tempo_sequences.append(tempos)
    
    # Pad sequences
    note_sequences_padded = pad_sequences(note_sequences, padding='post', maxlen=1000)
    chord_sequences_padded = pad_sequences(chord_sequences, padding='post', maxlen=1000)
    tempo_sequences_padded = pad_sequences(tempo_sequences, padding='post', maxlen=100)
    
    # Combine all features into a single feature array
    features = np.concatenate([note_sequences_padded, chord_sequences_padded, tempo_sequences_padded], axis=1)
    
    return features

In [7]:
train_features = extract_features(train_midi_files + augmented_train_midi_files)
dev_features = extract_features(dev_midi_files)
test_features = extract_features(test_midi_files)

In [8]:
# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = to_categorical(label_encoder.fit_transform(train_labels + train_labels))
dev_labels_encoded = to_categorical(label_encoder.transform(dev_labels))
test_labels_encoded = to_categorical(label_encoder.transform(test_labels))

In [9]:
# Define the LSTM model with dropout
def create_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(128))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
# Define the CNN model with dropout
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
# Input shape and number of classes
input_shape = (train_features.shape[1], 1)
num_classes = len(label_encoder.classes_)

In [12]:
# Reshape features for CNN model
train_features_cnn = train_features.reshape(train_features.shape[0], train_features.shape[1], 1)
dev_features_cnn = dev_features.reshape(dev_features.shape[0], dev_features.shape[1], 1)
test_features_cnn = test_features.reshape(test_features.shape[0], test_features.shape[1], 1)

In [13]:
# Create models
lstm_model = create_lstm_model(input_shape, num_classes)
cnn_model = create_cnn_model(input_shape, num_classes)

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Train models
lstm_model.fit(train_features, train_labels_encoded, validation_data=(dev_features, dev_labels_encoded), epochs=30, batch_size=64)

Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.1696 - loss: 2.1122 - val_accuracy: 0.2000 - val_loss: 2.0406
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.2398 - loss: 1.9532 - val_accuracy: 0.2000 - val_loss: 2.0762
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.2395 - loss: 1.9501 - val_accuracy: 0.2286 - val_loss: 2.0801
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.3055 - loss: 1.8195 - val_accuracy: 0.2571 - val_loss: 2.0732
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - accuracy: 0.2986 - loss: 1.8372 - val_accuracy: 0.2000 - val_loss: 2.0902
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - accuracy: 0.2608 - loss: 1.8613 - val_accuracy: 0.2571 - val_loss: 1.9837
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - accuracy: 0.3230 - loss: 1.7272 - val_accuracy: 0.2571 - val_loss: 2.0265
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.3260 - loss: 1.7143 - val_accuracy: 0.3143 - val_loss:

In [15]:
# Evaluate models
lstm_eval = lstm_model.evaluate(test_features, test_labels_encoded)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step - accuracy: 0.4905 - loss: 1.5268


In [16]:
print(f"LSTM Model - Loss: {lstm_eval[0]}, Accuracy: {lstm_eval[1]}")

LSTM Model - Loss: 1.5501371622085571, Accuracy: 0.48571428656578064


In [17]:
cnn_model.fit(train_features_cnn, train_labels_encoded, validation_data=(dev_features_cnn, dev_labels_encoded), epochs=30, batch_size=64)

Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.1372 - loss: 347.1519 - val_accuracy: 0.1714 - val_loss: 394.4645
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.1692 - loss: 288.0038 - val_accuracy: 0.2571 - val_loss: 127.5615
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.2920 - loss: 90.5766 - val_accuracy: 0.4000 - val_loss: 50.4788
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4706 - loss: 35.5999 - val_accuracy: 0.4571 - val_loss: 38.2517
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5311 - loss: 28.0079 - val_accuracy: 0.3143 - val_loss: 35.3115
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.6072 - loss: 22.8701 - val_accuracy: 0.4857 - val_loss: 24.5140
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6650 - loss: 16.2105 - val_accuracy: 0.5429 - val_loss: 33.8156
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6977 - loss: 15.6298 - val_a

In [18]:
# Evaluate models
cnn_eval = cnn_model.evaluate(test_features_cnn, test_labels_encoded)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3917 - loss: 23.6624 


In [19]:
print(f"CNN Model - Loss: {cnn_eval[0]}, Accuracy: {cnn_eval[1]}")

CNN Model - Loss: 23.7609806060791, Accuracy: 0.4000000059604645
